# Assignment 3: Clustering

Szymon Siemieniuk 151947

Nikita Makarevich 153989

In [2]:
import sklearn
import numpy as np
import pandas as pd

df_data = pd.read_csv('data/data.csv')

In [3]:
df_data.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28
0,0,-0.389420,-0.912791,0.648951,0.589045,-0.830817,0.733624,2.258560,2,13,...,-0.478412,-0.757002,-0.763635,-1.090369,1.142641,-0.884274,1.137896,1.309073,1.463002,0.813527
1,1,-0.689249,-0.453954,0.654175,0.995248,-1.653020,0.863810,-0.090651,2,3,...,-0.428791,-0.089908,-1.784204,-0.839474,0.459685,1.759412,-0.275422,-0.852168,0.562457,-2.680541
2,2,0.809079,0.324568,-1.170602,-0.624491,0.105448,0.783948,1.988301,5,11,...,-0.413534,-1.602377,1.190984,3.267116,-0.088322,-2.168635,-0.974989,1.335763,-1.110655,-3.630723
3,3,-0.500923,0.229049,0.264109,0.231520,0.415012,-1.221269,0.138850,6,2,...,0.619283,1.287801,0.532837,1.036631,-2.041828,1.440490,-1.900191,-0.630771,-0.050641,0.238333
4,4,-0.671268,-1.039533,-0.270155,-1.830264,-0.290108,-1.852809,0.781898,8,7,...,-1.628830,-0.434948,0.322505,0.284326,-2.438365,1.473930,-1.044684,1.602686,-0.405263,-1.987263


In [7]:
cat_features = [
    'f_07',
    'f_08',
    'f_09',
    'f_10',
    'f_11',
    'f_12',
    'f_13'
]
df_cat = df_data[cat_features]

num_features = list(set(df_data.columns) - set(cat_features) - set('id'))
df_num = df_data[num_features]

columns = num_features
columns.extend(cat_features)

df_data = df_data[columns]

In [5]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

scaler = MinMaxScaler().fit(df_num)
norm = Normalizer().fit(df_cat)

df_scaled = pd.DataFrame(scaler.transform(df_num), columns=df_num.columns)

df_scaled[df_cat.columns] = norm.transform(df_cat)

df_scaled.drop(columns=['id'], inplace=True)

In [6]:
df_scaled

,f_04,f_28,f_19,f_20,f_00,f_02,f_25,f_26,f_24,f_27,...,f_05,f_16,f_17,f_07,f_08,f_09,f_10,f_11,f_12,f_13
0,0.431504,0.557570,0.467078,0.435327,0.470880,0.562361,0.546854,0.603894,0.450930,0.605064,...,0.558689,0.629554,0.449354,0.079368,0.515889,0.555573,0.198419,0.515889,0.238103,0.238103
1,0.335748,0.306763,0.472326,0.508380,0.438371,0.562946,0.446892,0.453613,0.623499,0.543135,...,0.573136,0.518967,0.510143,0.095564,0.143346,0.286691,0.191127,0.286691,0.764510,0.430037
2,0.540544,0.238558,0.473940,0.342750,0.600831,0.358769,0.397413,0.605750,0.367093,0.428078,...,0.564273,0.543429,0.480574,0.236757,0.520865,0.236757,0.378811,0.426162,0.142054,0.520865
3,0.576596,0.516282,0.583177,0.659253,0.458790,0.519301,0.331975,0.469007,0.602681,0.500973,...,0.341746,0.372635,0.338765,0.297775,0.099258,0.645179,0.397033,0.446663,0.297775,0.198517
4,0.494476,0.356527,0.345402,0.470595,0.440320,0.459521,0.392484,0.624310,0.604864,0.476586,...,0.271661,0.373856,0.564735,0.382255,0.334473,0.238909,0.143346,0.047782,0.621164,0.525600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97995,0.493436,0.531120,0.486993,0.489957,0.538866,0.412625,0.451594,0.410276,0.649153,0.524980,...,0.513801,0.381391,0.560797,0.290129,0.406181,0.464207,0.290129,0.406181,0.348155,0.406181
97996,0.486603,0.551405,0.530125,0.670252,0.548093,0.552702,0.548608,0.522255,0.546457,0.572857,...,0.497475,0.605012,0.665245,0.151330,0.453990,0.100887,0.252217,0.151330,0.554877,0.605320
97997,0.550976,0.545220,0.427752,0.322461,0.486066,0.700789,0.480162,0.661859,0.606804,0.617652,...,0.415183,0.574925,0.559543,0.431331,0.485247,0.539164,0.377415,0.269582,0.215666,0.161749
97998,0.582414,0.494422,0.516531,0.640757,0.546869,0.574939,0.428321,0.400851,0.588578,0.462079,...,0.484191,0.411495,0.532654,0.048679,0.389434,0.535472,0.535472,0.146038,0.438113,0.243396
